# Analyse von nach realen Personen benannte Straßen

In [6]:
# Bibliotheken laden
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import math
import seaborn as sns
from shapely import wkt
from geopy.distance import geodesic
import numpy as np
import geopandas as gpd
import requests
import folium
import re 
import branca
from branca.element import Element
import contextily as ctx

In [7]:
# CSV-Datei einlesen
file_path_persons = '../../data/clean/Data_Masterarbeit - list_named_after_humans.csv'
file_path_all_streets = '../../data/clean/Data_Masterarbeit - list_complete.csv'
file_path_street_layout = "../../data/raw/lhd_strassenzuege.csv"

df_persons = pd.read_csv(file_path_persons, sep=",")
df_all = pd.read_csv(file_path_all_streets, sep=",")
df_streets = pd.read_csv(file_path_street_layout, sep=";")

## Verteilung Wirkungsbereich und Tätigkeitsfeld (II)

In [ ]:
# HILFSFUNKTION
def plot_horizontal_bar(ax, counts_series, color_dict, title, total_for_percentage=None, axis_max_limit=None, label_order="pct_first"):
    """
    Plottet ein horizontales Balkendiagramm auf einer gegebenen Achse (ax).
    Zeigt prozentuale Anteile am Balkenende an.
    Verwendet 'axis_max_limit' für konsistente 'xlim' bei geteilten Achsen.
    """
    # Standard-Total für Prozentberechnung
    if total_for_percentage is None:
        total_for_percentage = counts_series.sum()

    # Fall 1: Keine Daten
    if counts_series.empty or total_for_percentage == 0:
        ax.set_title(f"{title}\n(Keine Daten)", fontsize=12)
        ax.axis("off")
        return

    # Daten für Plot umkehren
    categories = counts_series.index[::-1]
    values = counts_series.values[::-1]
    
    # Fall 2: Daten sind leer nach Filterung
    if values.size == 0:
        ax.set_title(f"{title}\n(Keine Daten)", fontsize=12)
        ax.axis("off")
        return

    # Prozentwerte berechnen
    percentages = (values / total_for_percentage) * 100
    colors = [color_dict.get(cat, "grey") for cat in categories]

    # Balken plotten
    ax.barh(categories, values, color=colors)
    ax.set_title(title, fontsize=14)
    ax.set_xlabel("Anzahl")

    # Achsenlimit-Logik
    if axis_max_limit is not None and axis_max_limit > 0:
        # Nimm das globale Maximum (übergeben für sharex=True Plots)
        max_val_for_limit = axis_max_limit
    else:
        # Lokales Maximum (für Einzelplots)
        max_val_for_limit = max(values)

    # Puffer rechts für Text
    right_padding_factor = 1.35 # 25% Puffer
    ax.set_xlim(right=max_val_for_limit * right_padding_factor)

    # Text-Puffer
    text_buffer_padding = max_val_for_limit * 0.01

    # Prozent-Labels hinzufügen (KORRIGIERTER TEIL)
    for i, (v, pct) in enumerate(zip(values, percentages)):
        
        # Diese Logik hat gefehlt:
        if label_order == 'abs_first':
            text_label = f"{v} ({pct:.1f}%)"
        else:
            text_label = f"{pct:.1f}% ({v})"
            
        ax.text(v + text_buffer_padding,
                i,
                text_label,  
                va='center',
                ha='left',
                fontsize=12)
    
    ax.tick_params(axis='y', labelsize=12)


# HAUPTSKRIPT

# Globale Definitionen
genders = ["männlich", "weiblich"]
column_gender = "Geschlecht"


# 2. Analyse: Wirkungsbereich & Tätigkeitsfeld
columns_to_analyze_1 = ["Wirkungsbereich", "Tätigkeitsfeld"]

for column in columns_to_analyze_1:
    print(f"  Verarbeite Spalte: {column}")
    
    # 1. Konsistente Farben sicherstellen
    all_categories = df_persons[column].dropna().unique()
    colors = plt.cm.tab20.colors
    color_dict = {cat: colors[i % len(colors)] for i, cat in enumerate(all_categories)}

    # Part A: Alle Geschlechter (Gesamt)
    category_counts_total = df_persons[column].value_counts()
    total_count_for_percentage_total = category_counts_total.sum()
    
    num_categories_total = len(category_counts_total)
    fig_height_total = max(8, num_categories_total * 0.5)
    
    fig, ax = plt.subplots(figsize=(10, fig_height_total))
    
    plot_horizontal_bar(
        ax,
        category_counts_total,
        color_dict,
        f"Verteilung {column} (Gesamt)",
        total_for_percentage=total_count_for_percentage_total
    )
    
    plt.tight_layout()
    plt.savefig(f"../../results/charts/bar_charts/{column}_gesamt_balkendiagramm.png", dpi=300)
    plt.close(fig)

    # Part B: Getrennt nach männlich und weiblich
    gender_counts_list = []
    global_max_val = 0
    for gender in genders:
        df_subset = df_persons[df_persons[column_gender] == gender]
        counts = df_subset[column].value_counts()
        gender_counts_list.append(counts)
        if not counts.empty:
            global_max_val = max(global_max_val, counts.max())

    max_num_categories_gender = 0
    if gender_counts_list:
        max_num_categories_gender = max(len(counts) for counts in gender_counts_list)
        
    subplot_height = max(6, max_num_categories_gender * 0.4)
    
    fig, axes = plt.subplots(1, 2, figsize=(18, subplot_height), sharex=True)
    
    for ax, gender, counts_series in zip(axes, genders, gender_counts_list):
        total_count_for_percentage_gender = counts_series.sum()
        
        plot_horizontal_bar(
            ax,
            counts_series,
            color_dict,
            f"{column} ({gender})",
            total_for_percentage=total_count_for_percentage_gender,
            axis_max_limit=global_max_val
        )

    plt.suptitle(f"Verteilung {column} nach Geschlecht", fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(f"../../results/charts/bar_charts/{column}_nach_geschlecht_balkendiagramm.png", dpi=400)
    plt.close(fig)

print("Analyse 1 abgeschlossen.")

# 3. Analyse: Detailliertes Tätigkeitsfeld II

print("\nStarte Analyse 2: Detailliertes Tätigkeitsfeld II...")

# Vorbereitung: Leere 'Tätigkeitsfeld II'-Felder füllen
df_persons['Tätigkeitsfeld II'] = df_persons['Tätigkeitsfeld II'].fillna('(leer)')

# Iteration über die *Hauptkategorien* (mit df_persons)
main_categories = df_persons['Tätigkeitsfeld'].dropna().unique()

for cat in main_categories:
    print(f"  Verarbeite Hauptkategorie: {cat}")
    
    # Filtere das DF für die aktuelle Hauptkategorie
    filtered_df = df_persons[df_persons['Tätigkeitsfeld'] == cat]

    # Konsistente Farbzuordnung (basierend auf Unterkategorie)
    alle_kategorien_sub = filtered_df['Tätigkeitsfeld II'].dropna().unique()
    colors = plt.cm.tab20.colors
    color_dict = {k: colors[i % len(colors)] for i, k in enumerate(alle_kategorien_sub)}

    
    # Part A: Globales Balkendiagramm (alle Geschlechter zusammen)
    category_counts_total = filtered_df['Tätigkeitsfeld II'].value_counts()
    total_count_for_percentage_total = category_counts_total.sum()

    num_categories_total = len(category_counts_total)
    fig_height_total = max(8, num_categories_total * 0.5)
    
    fig, ax = plt.subplots(figsize=(10, fig_height_total))
    
    plot_horizontal_bar(
        ax,
        category_counts_total,
        color_dict,
        f"Verteilung {cat} (Gesamt)",
        total_for_percentage=total_count_for_percentage_total,
        label_order="pct_first"
    )
    
    plt.tight_layout()
    # Dateiname anpassen (z.B. Leerzeichen ersetzen)
    safe_cat_name = str(cat).replace(' ', '_').replace('/', '-')
    plt.savefig(f"../../results/charts/bar_charts/{safe_cat_name}_gesamt_balkendiagramm.png", dpi=300) 
    plt.close(fig)

    
    # Part B: Nach Geschlecht getrennt
    gender_counts_list = []
    global_max_val = 0
    for g in genders:
        df_subset = filtered_df[filtered_df[column_gender] == g] 
        counts = df_subset['Tätigkeitsfeld II'].value_counts()
        gender_counts_list.append(counts)
        if not counts.empty:
            global_max_val = max(global_max_val, counts.max())

    max_num_categories_gender = 0
    if gender_counts_list:
        max_num_categories_gender = max(len(counts) for counts in gender_counts_list)
        
    subplot_height = max(6, max_num_categories_gender * 0.4)
    
    fig, axes = plt.subplots(1, 2, figsize=(18, subplot_height), sharex=True)
    
    for ax, g, counts_series in zip(axes, genders, gender_counts_list):
        total_count_for_percentage_gender = counts_series.sum()
        
        plot_horizontal_bar(
            ax,
            counts_series,
            color_dict,
            f"{cat} ({g})",
            total_for_percentage=total_count_for_percentage_gender,
            axis_max_limit=global_max_val
        )

    plt.suptitle(f"Verteilung {cat} nach Geschlecht", fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
   
    plt.savefig(f"../../results/charts/bar_charts/{safe_cat_name}_geschlecht_balkendiagramm.png", dpi=300)
    plt.close(fig)

print("Analyse 2 abgeschlossen.")
print("\nAlle Analysen erfolgreich durchgeführt.")

  Verarbeite Spalte: Wirkungsbereich
  Verarbeite Spalte: Tätigkeitsfeld
Analyse 1 abgeschlossen.

Starte Analyse 2: Detailliertes Tätigkeitsfeld II...
  Verarbeite Hauptkategorie: kulturell
  Verarbeite Hauptkategorie: politisch
  Verarbeite Hauptkategorie: ökonomisch
  Verarbeite Hauptkategorie: wissenschaftlich
  Verarbeite Hauptkategorie: religiös
  Verarbeite Hauptkategorie: administrativ
  Verarbeite Hauptkategorie: militärisch
Analyse 2 abgeschlossen.

Alle Analysen erfolgreich durchgeführt.


## Distanzanalyse

In [9]:
# WKT in Geometrie umwandeln
def safe_wkt(val):
    if pd.isna(val) or str(val).strip() == "":
        return None
    try:
        return wkt.loads(str(val).replace("Point", "POINT"))
    except:
        return None

df_all["geometry"] = df_all["Koordinaten [WD]"].apply(safe_wkt)
df_all["lon"] = df_all["geometry"].apply(lambda g: g.x if g else np.nan)
df_all["lat"] = df_all["geometry"].apply(lambda g: g.y if g else np.nan)

# Merge über ID
df = pd.merge(df_persons, df_all, on="ID", how="inner")

# Altmarkt Dresden als Zentrum
center = (51.0504, 13.7373)

# Distanz berechnen
print("Berechne 'distanz_km'...") # Statusmeldung
df["distanz_km"] = np.nan
mask = df["lat"].notna() & df["lon"].notna()
df.loc[mask, "distanz_km"] = df.loc[mask].apply(
    lambda row: geodesic((row["lat"], row["lon"]), center).kilometers,
    axis=1
)

# Wähle die Spalte aus, die analysiert werden soll
kategorie_spalte = "Tätigkeitsfeld_x" 

# Daten filtern (NaNs in Distanz ODER Kategorie entfernen)
plot_data = df[df["distanz_km"].notna() & df[kategorie_spalte].notna()]

# Finde die Top 15 Kategorien (damit der Plot lesbar bleibt)
top_kategorien = plot_data[kategorie_spalte].value_counts().nlargest(15).index

# Filtere die Daten auf diese Top-Kategorien
plot_data = plot_data[plot_data[kategorie_spalte].isin(top_kategorien)]

# Reihenfolge der Kategorien (nach Median-Distanz sortiert)
order = plot_data.groupby(kategorie_spalte)["distanz_km"] \
                 .median() \
                 .sort_values() \
                 .index

# Statistische Kennzahlen je Kategorie
stats = plot_data.groupby(kategorie_spalte)["distanz_km"] \
                 .agg(["mean", "median", "count"]) \
                 .reindex(order) # .reindex() stellt sicher, dass 'stats' dieselbe Reihenfolge hat wie 'order'

custom_color_map = {
    'kulturell': '#1f77b4',       # (Dunkelblau)
    'politisch': "#aec7e8",       # (Hellblau)
    'wissenschaftlich': '#ffdead', # (Helles Orange / 'navajowhite')
    'ökonomisch': "#ff7300",       # (Orange)
    'religiös': '#2ca02c',      # (Grün)
    'administrativ': '#98df8a',      # (Helles Grün)
    'militärisch': '#d62728'      # (Rot)
}

ordered_colors = [custom_color_map.get(cat, 'grey') for cat in order]

# Boxplot zeichnen
plt.figure(figsize=(12, 10)) 
sns.boxplot(
    data=plot_data,       
    x=kategorie_spalte,   
    y="distanz_km",
    palette=custom_color_map,
    order=order           
)
plt.title(f"Distanz von Straßen nach 'Tätigkeitsfeld' zum Altmarkt (Dresden)")
plt.xlabel("Tätigkeitsfeld")
plt.ylabel("Distanz zum Zentrum (km)")

# Dreht die x-Achsen-Beschriftungen, damit sie lesbar sind
plt.xticks(rotation=75, ha='right', fontsize=13)

# Mittelwert & Median über jeder Box eintragen
for i, (kategorie, row) in enumerate(stats.iterrows()):
    mean_val = row["mean"]
    median_val = row["median"]
    plt.text(
        i,
        mean_val + 0.2, # Position leicht über dem Mittelwert
        f"Ø {mean_val:.2f}\nMed {median_val:.2f}",
        ha="center",
        color="black",
        fontsize=12,
        bbox=dict(facecolor="white", alpha=0.7, edgecolor="none", pad=0.1)
    )

# Legende für Erklärung
plt.legend(
    [plt.Line2D([0], [0], color="white")],
    ["Ø = Mittelwert | Med = Median"],
    loc="upper left",
    frameon=False
)


plt.tight_layout() 

# Diagramm speichern
plt.savefig("../../results/charts/boxplots/distanzanalyse_taetigkeitsfeld_boxplot.png", dpi=300)
plt.close()

Berechne 'distanz_km'...


C:\Users\juliu\AppData\Local\Temp\ipykernel_9248\1935044501.py:66: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(


In [10]:
def safe_wkt(val):
    if pd.isna(val) or str(val).strip() == "":
        return None
    try:
        return wkt.loads(str(val).replace("Point", "POINT"))
    except:
        return None

df_all["geometry"] = df_all["Koordinaten [WD]"].apply(safe_wkt)
df_all["lon"] = df_all["geometry"].apply(lambda g: g.x if g else np.nan)
df_all["lat"] = df_all["geometry"].apply(lambda g: g.y if g else np.nan)
df = pd.merge(df_persons, df_all, on="ID", how="inner")
center = (51.0504, 13.7373)

# Distanz berechnen
df["distanz_km"] = np.nan
mask = df["lat"].notna() & df["lon"].notna()
df.loc[mask, "distanz_km"] = df.loc[mask].apply(
    lambda row: geodesic((row["lat"], row["lon"]), center).kilometers,
    axis=1
)
print("Datenvorbereitung und Distanzberechnung abgeschlossen.")


# 1. VARIABLEN DEFINIEREN
haupt_kategorie_spalte = "Tätigkeitsfeld_x"
sub_kategorie_spalte = "Tätigkeitsfeld II_x" 

# BITTE ANPASSEN, welches Haupt-Feld analysiert soll?
zu_analysierendes_feld = "wissenschaftlich"

print(f"Starte Analyse für Hauptkategorie: '{zu_analysierendes_feld}'")

# 2. DATENFILTERUNG
plot_data = df[
    (df["distanz_km"].notna()) &                                  
    (df[sub_kategorie_spalte].notna()) &                           
    (df[haupt_kategorie_spalte] == zu_analysierendes_feld) 
]

# 3. TOP KATEGORIEN (jetzt für die Sub-Kategorie)
top_sub_kategorien = plot_data[sub_kategorie_spalte].value_counts().nlargest(14).index
plot_data = plot_data[plot_data[sub_kategorie_spalte].isin(top_sub_kategorien)]

# 4. REIHENFOLGE & STATISTIKEN (für die Sub-Kategorie)
if not plot_data.empty:
    order = plot_data.groupby(sub_kategorie_spalte)["distanz_km"] \
                     .median() \
                     .sort_values() \
                     .index
else:
    order = [] # Leere Liste, falls keine Daten vorhanden

stats = plot_data.groupby(sub_kategorie_spalte)["distanz_km"] \
                 .agg(["mean", "median", "count"]) \
                 .reindex(order)

# 5. BOXPLOT ZEICHNEN
plt.figure(figsize=(12, 10)) 
sns.boxplot(
    data=plot_data,
    x=sub_kategorie_spalte,
    y="distanz_km",
    #palette="pastel",
    order=order
)

# Titel und Labels
plt.title(f"Distanz-Analyse für '{zu_analysierendes_feld}' (Top 14 Unterkategorien)")
plt.xlabel(f"Unterkategorie")
plt.ylabel("Distanz zum Zentrum (km)")
plt.xticks(rotation=75, ha='right', fontsize=13)

# Mittelwert & Median über jeder Box eintragen
for i, (kategorie, row) in enumerate(stats.iterrows()):
    mean_val = row["mean"]
    median_val = row["median"]
    plt.text(
        i, mean_val + 0.2, f"Ø {mean_val:.2f}\nMed {median_val:.2f}",
        ha="center", color="black", fontsize=12,
        bbox=dict(facecolor="white", alpha=0.7, edgecolor="none", pad=0.1)
    )

plt.legend(
    [plt.Line2D([0], [0], color="white")],
    ["Ø = Mittelwert | Med = Median"],
    loc="upper left", frameon=False
)

plt.tight_layout() 

# Ersetzt ungültige Zeichen im Dateinamen
sicherer_dateiname = zu_analysierendes_feld.replace(' ', '_').replace('/', '-').lower()
plt.savefig(f"../../results/charts/boxplots/distanzanalyse_subkategorie_{sicherer_dateiname}_boxplot.png", dpi=300)
plt.close()

Datenvorbereitung und Distanzberechnung abgeschlossen.
Starte Analyse für Hauptkategorie: 'wissenschaftlich'


## Stadtbezirke

### Folium Karte Tätigkeitsfeld

In [11]:
def parse_wkt_safe(x):
    if pd.isna(x):
        return None
    try:
        return wkt.loads(str(x))
    except:
        return None

df_streets["geom_clean"] = df_streets["geom"].astype(str).str.replace(r"SRID=\d+;", "", regex=True)
df_streets["geometry"] = df_streets["geom_clean"].apply(parse_wkt_safe)
strassen_gdf = gpd.GeoDataFrame(df_streets.dropna(subset=["geometry"]), geometry="geometry", crs="EPSG:4326")
strassen_gdf["strasse_key"] = strassen_gdf["strasse"].str.strip().str.lower()


df_persons["strasse_key"] = df_persons["Straßenname"].str.strip().str.lower()

# Merge: Straßen + Personen-Daten
strassen_merge = strassen_gdf.merge(
    df_persons[["strasse_key", "Tätigkeitsfeld"]], # Nur die benötigten Spalten
    left_on="strasse_key",
    right_on="strasse_key",
    how="left"
)

# 3. Stadtbezirke laden (GeoJSON)
stadtbezirke_url = "https://kommisdd.dresden.de/net4/public/ogcapi/collections/L139/items"
try:
    stadt_json = requests.get(stadtbezirke_url).json()
    stadt_gdf = gpd.GeoDataFrame.from_features(stadt_json["features"])
    # Prüfen, ob CRS fehlt, und DANN erst setzen
    if stadt_gdf.crs is None:
        stadt_gdf = stadt_gdf.set_crs(epsg=4326)
    # Sicherstellen, dass es 4326 ist (falls es in anderem CRS geladen wurde)
    stadt_gdf = stadt_gdf.to_crs(epsg=4326) 
    print("Stadtbezirke erfolgreich geladen.")
except Exception as e:
    print(f"Fehler beim Laden der Stadtbezirke: {e}")


# 4. Räumliche Verknüpfung

# 1: Erst filtern, dann joinen
# Nur Straßen filtern, die überhaupt ein Tätigkeitsfeld haben
personen_strassen_gdf = strassen_merge.dropna(subset=["Tätigkeitsfeld"])

# CRS-Check
if personen_strassen_gdf.crs != stadt_gdf.crs:
    print("Warnung: CRS stimmen nicht überein. Projiziere strassen_gdf neu...")
    personen_strassen_gdf = personen_strassen_gdf.to_crs(stadt_gdf.crs)

# sjoin nur mit den relevanten Straßen durchführen
strassen_bezirk = gpd.sjoin(personen_strassen_gdf, stadt_gdf, how="left", predicate="intersects")
print("Räumliche Verknüpfung (sjoin) abgeschlossen.")


# VORBEREITUNG FÜR DIE SCHLEIFE
all_types = strassen_bezirk["Tätigkeitsfeld"].dropna().unique()
print(f"Gefundene Tätigkeitsfelder zum Iterieren: {all_types}")

# Gesamtanzahl (Nenner)
# Wichtig: drop_duplicates, falls eine Straße mehrere Bezirke schneidet
personen_strassen_unique = strassen_bezirk.drop_duplicates(subset=["bez", "strasse_key"])
gesamt_pro_bezirk = personen_strassen_unique.groupby("bez").size()

# Kartenmittelpunkt
center = stadt_gdf.geometry.centroid.unary_union.centroid
print("Starte Kartenerstellung...")

# START: Iteration durch alle Benennungstypen-

for ausgewaehltes_feld in all_types:
    
    print(f"\n--- Verarbeite Kategorie: {ausgewaehltes_feld} ---")

    # 5. Straßen für aktuelles Feld filtern
    strassen_feld = strassen_bezirk[strassen_bezirk["Tätigkeitsfeld"] == ausgewaehltes_feld]
    
    # 6. Prozentualer Anteil pro Bezirk berechnen
    strassen_feld_unique = strassen_feld.drop_duplicates(subset=["bez", "strasse_key"])
    feld_counts = strassen_feld_unique.groupby("bez").size()
    
    anteil_feld = (feld_counts / gesamt_pro_bezirk).fillna(0).reset_index()
    anteil_feld.columns = ["bez", "anteil_feld"]
    stadt_gdf_feld = stadt_gdf.merge(anteil_feld, on="bez", how="left")
    stadt_gdf_feld["anteil_feld"] = stadt_gdf_feld["anteil_feld"].fillna(0)

    # 7. Karte erstellen
    karte = folium.Map(location=[center.y, center.x], zoom_start=12)


    title_text = f"Verteilung des Tätigkeitsfeldes: '{ausgewaehltes_feld}'"
    title_html = f"""
                 <h3 align="center" style="font-size:16px; font-family: 'Helvetica Neue', Helvetica, Arial, sans-serif;">
                 <b>{title_text}</b>
                 </h3>
                 """
    # Fügt die h3-Überschrift hinzu
    karte.get_root().html.add_child(Element(title_html))

    # Choropleth für Bezirke
    folium.Choropleth(
        geo_data=stadt_gdf_feld,
        data=stadt_gdf_feld,
        columns=["bez", "anteil_feld"],
        key_on="feature.properties.bez",
        fill_color="YlOrRd", 
        fill_opacity=0.7,     
        line_opacity=0.2,
        legend_name=f"Anteil Personen-'{ausgewaehltes_feld}' Straßen pro Bezirk"
    ).add_to(karte)

    # Tooltip
    folium.GeoJson(
        stadt_gdf_feld,
        tooltip=folium.GeoJsonTooltip(fields=["bez", "anteil_feld"],
                                        aliases=["Bezirk", f"Anteil '{ausgewaehltes_feld}' (%)"],
                                        localize=True,
                                        sticky=False),
        style_function=lambda x: {"fillOpacity": 0, "color": "black", "weight": 0.5}
    ).add_to(karte)

    # 8. Straßen einzeichne
    folium.GeoJson(
        strassen_feld,
        style_function=lambda x: {'color': '#00008B', 'weight': 2.5, 'opacity': 0.8}
    ).add_to(karte)

    # 9. Karte speichern & anzeigen
    safe_filename = re.sub(r'[^\w\-_\. ]', '_', ausgewaehltes_feld)
    filename = f"../../results/maps/Stadtbezirke/Karte_Stadtbezirk_Tätigkeitsfeld_{safe_filename}.html"
    
    karte.save(filename)
    
print("\n--- Alle Karten erfolgreich erstellt. ---")

Stadtbezirke erfolgreich geladen.
Räumliche Verknüpfung (sjoin) abgeschlossen.
Gefundene Tätigkeitsfelder zum Iterieren: ['politisch' 'religiös' 'kulturell' 'wissenschaftlich' 'ökonomisch'
 'militärisch' 'administrativ']


C:\Users\juliu\AppData\Local\Temp\ipykernel_9248\334874157.py:66: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = stadt_gdf.geometry.centroid.unary_union.centroid
C:\Users\juliu\AppData\Local\Temp\ipykernel_9248\334874157.py:66: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = stadt_gdf.geometry.centroid.unary_union.centroid


Starte Kartenerstellung...

--- Verarbeite Kategorie: politisch ---

--- Verarbeite Kategorie: religiös ---

--- Verarbeite Kategorie: kulturell ---

--- Verarbeite Kategorie: wissenschaftlich ---

--- Verarbeite Kategorie: ökonomisch ---

--- Verarbeite Kategorie: militärisch ---

--- Verarbeite Kategorie: administrativ ---

--- Alle Karten erfolgreich erstellt. ---


### Statisches PNG Tätigkeitsfeld

In [4]:
def parse_wkt_safe(x):
    if pd.isna(x):
        return None
    try:
        return wkt.loads(str(x))
    except:
        return None

df_streets["geom_clean"] = df_streets["geom"].astype(str).str.replace(r"SRID=\d+;", "", regex=True)
df_streets["geometry"] = df_streets["geom_clean"].apply(parse_wkt_safe)
strassen_gdf = gpd.GeoDataFrame(df_streets.dropna(subset=["geometry"]), geometry="geometry", crs="EPSG:4326")
strassen_gdf["strasse_key"] = strassen_gdf["strasse"].str.strip().str.lower()

# 2. Kategorie-Daten laden (df_persons)

df_persons["strasse_key"] = df_persons["Straßenname"].str.strip().str.lower()

# Merge: Straßen + Kategorie
strassen_merge = strassen_gdf.merge(
    df_persons[["strasse_key", "Tätigkeitsfeld"]], 
    left_on="strasse_key",
    right_on="strasse_key",
    how="left"
)

# 3. Stadtbezirke laden (GeoJSON)
stadtbezirke_url = "https://kommisdd.dresden.de/net4/public/ogcapi/collections/L139/items"
try:
    stadt_json = requests.get(stadtbezirke_url).json()
    stadt_gdf = gpd.GeoDataFrame.from_features(stadt_json["features"])
    if stadt_gdf.crs is None:
        stadt_gdf = stadt_gdf.set_crs(epsg=4326)
    stadt_gdf = stadt_gdf.to_crs(epsg=4326)
    print("Stadtbezirke erfolgreich geladen.")
except Exception as e:
    print(f"Fehler beim Laden der Stadtbezirke: {e}")

# 4. Räumliche Verknüpfung Straße ↔ Bezirk (Optimiert)

# Filter auf "Tätigkeitsfeld"
strassen_mit_typ = strassen_merge.dropna(subset=["Tätigkeitsfeld"])

if strassen_mit_typ.crs != stadt_gdf.crs:
    print("Warnung: CRS stimmen nicht überein. Projiziere strassen_gdf neu...")
    strassen_mit_typ = strassen_mit_typ.to_crs(stadt_gdf.crs)

strassen_bezirk = gpd.sjoin(strassen_mit_typ, stadt_gdf, how="left", predicate="intersects")
print("Räumliche Verknüpfung (sjoin) abgeschlossen.")

# VORBEREITUNG FÜR DIE SCHLEIFE

# Spalte "Tätigkeitsfeld"
all_types = strassen_bezirk["Tätigkeitsfeld"].dropna().unique()
personen_strassen = strassen_bezirk.dropna(subset=["Tätigkeitsfeld"])
personen_strassen_unique = personen_strassen.drop_duplicates(subset=["bez", "strasse_key"])
gesamt_pro_bezirk = personen_strassen_unique.groupby("bez").size()

center = stadt_gdf.geometry.centroid.unary_union.centroid

print("Starte Erstellung von statischen PNG-Karten...")
# START: Iteration durch alle Benennungstypen

for ausgewaehltes_feld in all_types:
    
    print(f"\n--- Verarbeite Kategorie: {ausgewaehltes_feld} ---")

    # 5. Filtern
    strassen_feld = strassen_bezirk[strassen_bezirk["Tätigkeitsfeld"] == ausgewaehltes_feld]

    # 6. Prozentrechnung
    strassen_feld_unique = strassen_feld.drop_duplicates(subset=["bez", "strasse_key"])
    feld_counts = strassen_feld_unique.groupby("bez").size()
    anteil_feld = (feld_counts / gesamt_pro_bezirk).fillna(0).reset_index()
    anteil_feld.columns = ["bez", "anteil_feld"]
    stadt_gdf_feld = stadt_gdf.merge(anteil_feld, on="bez", how="left")
    stadt_gdf_feld["anteil_feld"] = stadt_gdf_feld["anteil_feld"].fillna(0)

    # Spalte für die Legende (Werte von 0-100)
    stadt_gdf_feld["anteil_prozent"] = stadt_gdf_feld["anteil_feld"] * 100

    
    # 7. Daten für Plotting vorbereiten (CRS umwandeln)
    target_crs = "EPSG:3857"
    stadt_plot = stadt_gdf_feld.to_crs(target_crs)
    strassen_plot = strassen_feld.to_crs(target_crs) if not strassen_feld.empty else strassen_feld

    # 8. Karte erstellen
    fig, ax = plt.subplots(figsize=(12, 12)) 

    # A: Bezirke als Choropleth plotten
    stadt_plot.plot(
        ax=ax,
        column="anteil_prozent", 
        cmap="YlOrRd",          
        legend=True,          
        alpha=0.7,
        edgecolor='black',       
        linewidth=0.5,           
        legend_kwds={
            'label': f"Anteil '{ausgewaehltes_feld}' (%)",
            'shrink': 0.75, 
            'aspect': 30   
        }
    )
    
    # Legenden-Schriftgröße anpassen
    cax = fig.axes[-1] 
    cax.tick_params(labelsize=8) 
    cax.yaxis.label.set_size(9)  

    # B: Straßen-Linien plotten
    if not strassen_plot.empty:
        strassen_plot.plot(
            ax=ax,
            color="#00008B",        
            linewidth=1,         
            alpha=0.8           
        )

    # C: Hintergrundkarte hinzufügen
    ctx.add_basemap(
        ax,
        crs=target_crs,
        source=ctx.providers.OpenStreetMap.Mapnik,
        zoom=13
    )

    # 9. Karte finalisieren & als PNG speichern
    ax.set_axis_off() 
    plt.title(f"Verteilung Tätigkeitsfeld: {ausgewaehltes_feld}", fontsize=16)
    
    safe_filename = re.sub(r'[^\w\-_\. ]', '_', ausgewaehltes_feld)
    
    filename = f"../../results/images/Stadtbezirke/Karte_Stadtbezirk_Tätigkeitsfeld_{safe_filename}.png"
    
    plt.savefig(
        filename,
        dpi=300, 
        bbox_inches="tight" 
    )
    plt.close(fig) 

print("\n--- Alle statischen PNG-Karten (Tätigkeitsfeld) erfolgreich erstellt. ---")

Stadtbezirke erfolgreich geladen.
Räumliche Verknüpfung (sjoin) abgeschlossen.


C:\Users\juliu\AppData\Local\Temp\ipykernel_7252\1864864162.py:58: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = stadt_gdf.geometry.centroid.unary_union.centroid
C:\Users\juliu\AppData\Local\Temp\ipykernel_7252\1864864162.py:58: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = stadt_gdf.geometry.centroid.unary_union.centroid


Starte Erstellung von statischen PNG-Karten...

--- Verarbeite Kategorie: politisch ---

--- Verarbeite Kategorie: religiös ---

--- Verarbeite Kategorie: kulturell ---

--- Verarbeite Kategorie: wissenschaftlich ---

--- Verarbeite Kategorie: ökonomisch ---

--- Verarbeite Kategorie: militärisch ---

--- Verarbeite Kategorie: administrativ ---

--- Alle statischen PNG-Karten (Tätigkeitsfeld) erfolgreich erstellt. ---


In [5]:
# 1. Absolute Anzahl pro Bezirk & Tätigkeitsfeld
counts = personen_strassen_unique.groupby(["bez", "Tätigkeitsfeld"]).size().unstack(fill_value=0)

# Prozentualer Anteil (Werte bleiben 0.0 bis 1.0)
anteile = counts.div(counts.sum(axis=1), axis=0)

# 2. Gestapeltes Balkendiagramm
fig, ax = plt.subplots(figsize=(12, 8)) 

# Farben
farbe_map = {
    'kulturell': '#aec7e8',       # hellblau
    'politisch': '#98df8a',       # hellgrün
    'wissenschaftlich': '#ffbb78', # hellorange
    'ökonomisch': '#ff7f0e',    # dunkelorange
    'religiös': '#d62728',       # rot
    'administrativ': '#1f77b4',    # blau
    'militärisch': '#2ca02c'     # dunkelgrün
    # (Weitere Felder würden 'grey' erhalten)
}

color_list = [farbe_map.get(col, 'grey') for col in anteile.columns]


anteile.plot(kind="barh", stacked=True, color=color_list, ax=ax)


ax.set_xlabel("Anteil der Straßen in Prozent")
ax.set_ylabel("Bezirk")
ax.set_title("Prozentuale Anteile der Tätigkeitsfelder pro Bezirk")

# X-Achse als Prozent formatieren
ax.xaxis.set_major_formatter(ticker.PercentFormatter(xmax=1.0))

# 3. Absolute Zahl neben jedem Balken
for i, bezirk in enumerate(counts.index):
    total = counts.loc[bezirk].sum()
    ax.text(1.02, i, str(total), ha='left', va='center', fontweight='bold')

# X-Achsen-Limit anpassen, damit der Text für die Summe Platz hat
ax.set_xlim(right=1.15) 

plt.yticks(rotation=0)
plt.legend(title="Tätigkeitsfeld", bbox_to_anchor=(1.05, 1))
plt.tight_layout()

# 4. Diagramm speichern
plt.savefig(f"../../results/charts/bar_charts_district_comparison/Balkendiagramm_prozentuale_Anteile_Tätigkeitsfeld_Stadtbezirke_.png", dpi=300)
plt.close()

## Gemarkungen

### Folium Karte

In [12]:
def parse_wkt_safe(x):
    if pd.isna(x):
        return None
    try:
        return wkt.loads(str(x))
    except:
        return None

df_streets["geom_clean"] = df_streets["geom"].astype(str).str.replace(r"SRID=\d+;", "", regex=True)
df_streets["geometry"] = df_streets["geom_clean"].apply(parse_wkt_safe)
strassen_gdf = gpd.GeoDataFrame(df_streets.dropna(subset=["geometry"]), geometry="geometry", crs="EPSG:4326")
strassen_gdf["strasse_key"] = strassen_gdf["strasse"].str.strip().str.lower()


df_persons["strasse_key"] = df_persons["Straßenname"].str.strip().str.lower()

# Merge: Straßen + Personen-Daten
strassen_merge = strassen_gdf.merge(
    df_persons[["strasse_key", "Tätigkeitsfeld"]], # Nur die benötigten Spalten
    left_on="strasse_key",
    right_on="strasse_key",
    how="left"
)

# 3. Gemarkungen laden (GeoJSON)
stadtbezirke_url = "https://kommisdd.dresden.de/net4/public/ogcapi/collections/L73/items"
try:
    stadt_json = requests.get(stadtbezirke_url).json()
    stadt_gdf = gpd.GeoDataFrame.from_features(stadt_json["features"])
    if stadt_gdf.crs is None:
        stadt_gdf = stadt_gdf.set_crs(epsg=4326)
    stadt_gdf = stadt_gdf.to_crs(epsg=4326) 
    print("Gemarkungen erfolgreich geladen.")
except Exception as e:
    print(f"Fehler beim Laden der Gemarkungen: {e}")

# 4. Räumliche Verknüpfung
personen_strassen_gdf = strassen_merge.dropna(subset=["Tätigkeitsfeld"])

if personen_strassen_gdf.crs != stadt_gdf.crs:
    print("Warnung: CRS stimmen nicht überein. Projiziere strassen_gdf neu...")
    personen_strassen_gdf = personen_strassen_gdf.to_crs(stadt_gdf.crs)

strassen_bezirk = gpd.sjoin(personen_strassen_gdf, stadt_gdf, how="left", predicate="intersects")
print("Räumliche Verknüpfung (sjoin) abgeschlossen.")

# VORBEREITUNG FÜR DIE SCHLEIFE
all_types = strassen_bezirk["Tätigkeitsfeld"].dropna().unique()
print(f"Gefundene Tätigkeitsfelder zum Iterieren: {all_types}")

# Gesamtanzahl (Nenner)
personen_strassen_unique = strassen_bezirk.drop_duplicates(subset=["gem_nam", "strasse_key"])
gesamt_pro_bezirk = personen_strassen_unique.groupby("gem_nam").size()

# Kartenmittelpunkt
center = stadt_gdf.geometry.centroid.unary_union.centroid
print("Starte Kartenerstellung...")

# START: Iteration durch alle Benennungstypen

for ausgewaehltes_feld in all_types:
    
    print(f"\n--- Verarbeite Kategorie: {ausgewaehltes_feld} ---")

    # 5. Straßen für *aktuelles* Feld filtern
    strassen_feld = strassen_bezirk[strassen_bezirk["Tätigkeitsfeld"] == ausgewaehltes_feld]
    
    # 6. Prozentualer Anteil pro Bezirk berechnen
    strassen_feld_unique = strassen_feld.drop_duplicates(subset=["gem_nam", "strasse_key"])
    feld_counts = strassen_feld_unique.groupby("gem_nam").size()
    
    anteil_feld = (feld_counts / gesamt_pro_bezirk).fillna(0).reset_index()
    anteil_feld.columns = ["gem_nam", "anteil_feld"]
    stadt_gdf_feld = stadt_gdf.merge(anteil_feld, on="gem_nam", how="left")
    stadt_gdf_feld["anteil_feld"] = stadt_gdf_feld["anteil_feld"].fillna(0)

    # 7. Karte erstellen
    karte = folium.Map(location=[center.y, center.x], zoom_start=12)

    # Überschrift
    title_text = f"Verteilung des Tätigkeitsfeldes: '{ausgewaehltes_feld}'"
    title_html = f"""
                 <h3 align="center" style="font-size:16px; font-family: 'Helvetica Neue', Helvetica, Arial, sans-serif;">
                 <b>{title_text}</b>
                 </h3>
                 """
    karte.get_root().html.add_child(Element(title_html))

    # Choropleth für Bezirke
    folium.Choropleth(
        geo_data=stadt_gdf_feld,
        data=stadt_gdf_feld,
        columns=["gem_nam", "anteil_feld"], 
        key_on="feature.properties.gem_nam", 
        fill_color="YlOrRd", 
        fill_opacity=0.7,   
        line_opacity=0.2,
        legend_name=f"Anteil Personen-'{ausgewaehltes_feld}' Straßen pro Bezirk"
    ).add_to(karte)

    # Tooltip
    folium.GeoJson(
        stadt_gdf_feld,
        tooltip=folium.GeoJsonTooltip(
            fields=["gem_nam", "anteil_feld"],
            aliases=["Bezirk", f"Anteil '{ausgewaehltes_feld}' (%)"],
            localize=True,
            sticky=False),
        style_function=lambda x: {"fillOpacity": 0, "color": "black", "weight": 0.5}
    ).add_to(karte)

    # 8. Straßen einzeichnen
    folium.GeoJson(
        strassen_feld,
        style_function=lambda x: {'color': '#00008B', 'weight': 2.5, 'opacity': 0.8}
    ).add_to(karte)

    # 9. Karte speichern & anzeigen
    safe_filename = re.sub(r'[^\w\-_\. ]', '_', ausgewaehltes_feld)
    filename = f"../../results/maps/Gemarkungen/Karte_Gemarkung_Tätigkeitsfeld_{safe_filename}.html"
    
    karte.save(filename)
    
print("\n--- Alle Karten erfolgreich erstellt. ---")

Gemarkungen erfolgreich geladen.
Räumliche Verknüpfung (sjoin) abgeschlossen.
Gefundene Tätigkeitsfelder zum Iterieren: ['politisch' 'religiös' 'kulturell' 'wissenschaftlich' 'ökonomisch'
 'militärisch' 'administrativ']
Starte Kartenerstellung...

--- Verarbeite Kategorie: politisch ---


C:\Users\juliu\AppData\Local\Temp\ipykernel_9248\4089528513.py:56: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = stadt_gdf.geometry.centroid.unary_union.centroid
C:\Users\juliu\AppData\Local\Temp\ipykernel_9248\4089528513.py:56: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = stadt_gdf.geometry.centroid.unary_union.centroid



--- Verarbeite Kategorie: religiös ---

--- Verarbeite Kategorie: kulturell ---

--- Verarbeite Kategorie: wissenschaftlich ---

--- Verarbeite Kategorie: ökonomisch ---

--- Verarbeite Kategorie: militärisch ---

--- Verarbeite Kategorie: administrativ ---

--- Alle Karten erfolgreich erstellt. ---


### Statisches PNG Gemarkungen

In [7]:
def parse_wkt_safe(x):
    if pd.isna(x):
        return None
    try:
        return wkt.loads(str(x))
    except:
        return None

df_streets["geom_clean"] = df_streets["geom"].astype(str).str.replace(r"SRID=\d+;", "", regex=True)
df_streets["geometry"] = df_streets["geom_clean"].apply(parse_wkt_safe)
strassen_gdf = gpd.GeoDataFrame(df_streets.dropna(subset=["geometry"]), geometry="geometry", crs="EPSG:4326")
strassen_gdf["strasse_key"] = strassen_gdf["strasse"].str.strip().str.lower()


df_persons["strasse_key"] = df_persons["Straßenname"].str.strip().str.lower()

# Merge: Straßen + Kategorie
strassen_merge = strassen_gdf.merge(
    df_persons[["strasse_key", "Tätigkeitsfeld"]], 
    left_on="strasse_key",
    right_on="strasse_key",
    how="left"
)

# 3. Gemarkungen laden (GeoJSON)
stadtbezirke_url = "https://kommisdd.dresden.de/net4/public/ogcapi/collections/L73/items"
try:
    stadt_json = requests.get(stadtbezirke_url).json()
    stadt_gdf = gpd.GeoDataFrame.from_features(stadt_json["features"])
    if stadt_gdf.crs is None:
        stadt_gdf = stadt_gdf.set_crs(epsg=4326)
    stadt_gdf = stadt_gdf.to_crs(epsg=4326)
    print("Gemarkungen erfolgreich geladen.")
except Exception as e:
    print(f"Fehler beim Laden der Gemarkungen: {e}")

# 4. Räumliche Verknüpfung Straße ↔ Bezirk (Optimiert)
strassen_mit_typ = strassen_merge.dropna(subset=["Tätigkeitsfeld"])

if strassen_mit_typ.crs != stadt_gdf.crs:
    print("Warnung: CRS stimmen nicht überein. Projiziere strassen_gdf neu...")
    strassen_mit_typ = strassen_mit_typ.to_crs(stadt_gdf.crs)

strassen_bezirk = gpd.sjoin(strassen_mit_typ, stadt_gdf, how="left", predicate="intersects")
print("Räumliche Verknüpfung (sjoin) abgeschlossen.")

# VORBEREITUNG FÜR DIE SCHLEIFE
all_types = strassen_bezirk["Tätigkeitsfeld"].dropna().unique()

personen_strassen = strassen_bezirk.dropna(subset=["Tätigkeitsfeld"])

personen_strassen_unique = personen_strassen.drop_duplicates(subset=["gem_nam", "strasse_key"])
gesamt_pro_bezirk = personen_strassen_unique.groupby("gem_nam").size()

center = stadt_gdf.geometry.centroid.unary_union.centroid

print("Starte Erstellung von statischen PNG-Karten...")


# START: Iteration durch alle Benennungstypen

for ausgewaehltes_feld in all_types:
    
    print(f"\n--- Verarbeite Kategorie: {ausgewaehltes_feld} ---")

    # 5. Filtern
    strassen_feld = strassen_bezirk[strassen_bezirk["Tätigkeitsfeld"] == ausgewaehltes_feld]

    # 6. Prozentrechnung-
    strassen_feld_unique = strassen_feld.drop_duplicates(subset=["gem_nam", "strasse_key"])
    feld_counts = strassen_feld_unique.groupby("gem_nam").size()
    anteil_feld = (feld_counts / gesamt_pro_bezirk).fillna(0).reset_index()
    anteil_feld.columns = ["gem_nam", "anteil_feld"]
    stadt_gdf_feld = stadt_gdf.merge(anteil_feld, on="gem_nam", how="left")
    
    stadt_gdf_feld["anteil_feld"] = stadt_gdf_feld["anteil_feld"].fillna(0)
    stadt_gdf_feld["anteil_prozent"] = stadt_gdf_feld["anteil_feld"] * 100

    
    # 7. Daten für Plotting vorbereiten (CRS umwandeln)
    target_crs = "EPSG:3857"
    stadt_plot = stadt_gdf_feld.to_crs(target_crs)
    strassen_plot = strassen_feld.to_crs(target_crs) if not strassen_feld.empty else strassen_feld

    # 8. Karte erstellen
    fig, ax = plt.subplots(figsize=(12, 12)) 

    # A: Bezirke als Choropleth plotten
    stadt_plot.plot(
        ax=ax,
        column="anteil_prozent", 
        cmap="YlOrRd",        
        legend=True,          
        alpha=0.7,
        edgecolor='black',       
        linewidth=0.5,           
        legend_kwds={
            'label': f"Anteil '{ausgewaehltes_feld}' (%)", 
            'shrink': 0.75, 
            'aspect': 30   
        }
    )
    
    cax = fig.axes[-1] 
    cax.tick_params(labelsize=8) 
    cax.yaxis.label.set_size(9)  

    # B: Straßen-Linien plotten
    if not strassen_plot.empty:
        strassen_plot.plot(
            ax=ax,
            color="#00008B",    
            linewidth=1.0,       
            alpha=0.8           
        )

    # C: Hintergrundkarte hinzufügen
    ctx.add_basemap(
        ax,
        crs=target_crs,
        source=ctx.providers.OpenStreetMap.Mapnik,
        zoom=13
    )

    # 9. Karte finalisieren & als PNG speichern
    ax.set_axis_off() 
    plt.title(f"Verteilung Tätigkeitsfeld: {ausgewaehltes_feld}", fontsize=16)
    
    safe_filename = re.sub(r'[^\w\-_\. ]', '_', ausgewaehltes_feld)
    
    filename = f"../../results/images/Gemarkungen/Karte_Gemarkung_Tätigkeitsfeld_{safe_filename}.png"
    
    plt.savefig(
        filename,
        dpi=300, 
        bbox_inches="tight" 
    )
    plt.close(fig) 

print("\n--- Alle statischen PNG-Karten (Tätigkeitsfeld) erfolgreich erstellt. ---")

Gemarkungen erfolgreich geladen.
Räumliche Verknüpfung (sjoin) abgeschlossen.
Starte Erstellung von statischen PNG-Karten...

--- Verarbeite Kategorie: politisch ---


C:\Users\juliu\AppData\Local\Temp\ipykernel_7252\1722231796.py:55: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = stadt_gdf.geometry.centroid.unary_union.centroid
C:\Users\juliu\AppData\Local\Temp\ipykernel_7252\1722231796.py:55: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = stadt_gdf.geometry.centroid.unary_union.centroid



--- Verarbeite Kategorie: religiös ---

--- Verarbeite Kategorie: kulturell ---

--- Verarbeite Kategorie: wissenschaftlich ---

--- Verarbeite Kategorie: ökonomisch ---

--- Verarbeite Kategorie: militärisch ---

--- Verarbeite Kategorie: administrativ ---

--- Alle statischen PNG-Karten (Tätigkeitsfeld) erfolgreich erstellt. ---


In [ ]:
# 1. Absolute Anzahl pro Gemarkung & Tätigkeitsfeld
counts = personen_strassen_unique.groupby(["gem_nam", "Tätigkeitsfeld"]).size().unstack(fill_value=0)

# Prozentualer Anteil (Werte bleiben 0.0 bis 1.0)
anteile = counts.div(counts.sum(axis=1), axis=0)

# 2. Gestapeltes Balkendiagramm

# Berechnung der optimalen Höhe basierend auf der Anzahl der Gemarkungen
num_gemarkungen = len(anteile.index)
fig_height = max(6, num_gemarkungen * 0.5)

# Figure mit der berechneten Höhe erstellen
fig, ax = plt.subplots(figsize=(12, fig_height)) 

# Farben
farbe_map = {
    'kulturell': '#aec7e8',       # hellblau
    'politisch': '#98df8a',       # hellgrün
    'wissenschaftlich': '#ffbb78', # hellorange
    'ökonomisch': '#ff7f0e',    # dunkelorange
    'religiös': '#d62728',       # rot
    'administrativ': '#1f77b4',    # blau
    'militärisch': '#2ca02c'     # dunkelgrün
    # (Weitere Felder würden 'grey' erhalten)
}

color_list = [farbe_map.get(col, 'grey') for col in anteile.columns]

# Horizontal plotten (kind="barh")
anteile.plot(kind="barh", stacked=True, color=color_list, ax=ax)

ax.set_xlabel("Anteil der Straßen in Prozent") 
ax.set_ylabel("Gemarkung")
ax.set_title("Prozentuale Anteile der Tätigkeitsfelder pro Gemarkung")

# X-Achse als Prozent formatieren (von 0.0-1.0 zu 0%-100%)
ax.xaxis.set_major_formatter(ticker.PercentFormatter(xmax=1.0))


# 3. Absolute Zahl neben jedem Balken
for i, bezirk in enumerate(counts.index):
    total = counts.loc[bezirk].sum()
    ax.text(1.02, i, str(total), ha='left', va='center', fontweight='bold')

# X-Achsen-Limit anpassen, damit der Text für die Summe Platz hat
ax.set_xlim(right=1.15) 

plt.yticks(rotation=0)
plt.legend(title="Tätigkeitsfeld", bbox_to_anchor=(1.05, 1))
plt.tight_layout()

# 4. Diagramm speichern
plt.savefig(f"../../results/charts/bar_charts_district_comparison/Balkendiagramm_prozentuale_Anteile_Tätigkeitsfeld_Gemarkungen.png", dpi=300)
plt.close()

## Verteilung Verbindung und Partei

In [12]:
# Schritt 1: Leere Werte in relevanten Spalten mit "" ersetzen
for column in ["Verbindung", "Partei"]:
    df_persons[column] = df_persons[column].fillna("").astype(str)

gender = ["männlich", "weiblich"]

# Schritt 2: Für jede Spalte (Verbindung, Partei) Diagramme erstellen
for column in ["Verbindung", "Partei"]:
    # Alle Werte splitten
    split_values = df_persons[column].str.split(',').explode().str.strip()
    split_values = split_values[split_values != ""]  # leere rausfiltern

    # Konsistente Farbzuordnung
    all_categories = split_values.unique()
    colours = plt.cm.tab20.colors
    colour_dict = {k: colours[i % len(colours)] for i, k in enumerate(all_categories)}

    # 1. Globales Balkendiagramm
    category_counter = split_values.value_counts(ascending=True) 
    total_global = category_counter.sum()
    colour_list = [colour_dict[k] for k in category_counter.index]

    # Figure-Größe für potenziell viele Kategorien angepasst
    plt.figure(figsize=(12, max(8, len(category_counter) * 0.4)))

    bars = plt.barh(
        category_counter.index,
        category_counter.values,
        color="steelblue"
    )
    
    # Benutzerdefinierte Labels (Anzahl + Prozent) für das globale Diagramm
    custom_labels_global = []
    for count in category_counter.values:
        pct = (count / total_global) * 100
        custom_labels_global.append(f"{pct:.1f}% ({count})") 

    plt.bar_label(bars, labels=custom_labels_global, padding=3, fontsize=12) 

    plt.title(f"Verteilung {column} (gesamt)")
    plt.xlabel("Anzahl") 
    plt.yticks(fontsize=12)
    
    # X-Achsenlimit anpassen, um Labels nicht abzuschneiden
    # Extra-Platz hinzufügen (z.B. 10% des Maximalwerts)
    max_val_global = category_counter.max() if not category_counter.empty else 0
    plt.xlim(0, max_val_global * 1.15)
    
    plt.tight_layout() 
    plt.savefig(f"../../results/charts/bar_charts/{column}_gesamt.png", dpi=300)
    plt.close()

    # 2. Nach Geschlecht getrennt
    # Figure-Größe angepasst für bessere Lesbarkeit
    fig, axes = plt.subplots(1, 2, figsize=(20, max(10, len(all_categories) * 0.4)))

    # Maximum über alle Subplots hinweg bestimmen
    max_val_per_gender = 0
    for g in gender:
        df_subset = df_persons.loc[df_persons['Geschlecht'] == g]
        split_subset = df_subset[column].str.split(',').explode().str.strip()
        split_subset = split_subset[split_subset != ""]
        category_counter = split_subset.value_counts()
        if not category_counter.empty:
            max_val_per_gender = max(max_val_per_gender, category_counter.max())

    for ax, g in zip(axes, gender):
        df_subset = df_persons.loc[df_persons['Geschlecht'] == g]

        split_subset = df_subset[column].str.split(',').explode().str.strip()
        split_subset = split_subset[split_subset != ""]  # leere raus

        category_counter = split_subset.value_counts(ascending=True)
        total = category_counter.sum()

        if total == 0:
            ax.set_title(f"{column} ({g})\nKeine Daten")
            ax.axis("off")
            if max_val_per_gender > 0:
                 ax.set_xlim(0, max_val_per_gender * 1.15) 
            continue

        colour_list = [colour_dict.get(k, "grey") for k in category_counter.index]

        bars = ax.barh(
            category_counter.index,
            category_counter.values,
            color="steelblue"
        )
        
        custom_labels = []
        for count in category_counter.values:
            pct = (count / total) * 100
            custom_labels.append(f"{pct:.1f}% ({count})")

        ax.bar_label(bars, labels=custom_labels, padding=3)
        
        ax.set_title(f"{column} ({g})")
        ax.set_xlabel("Anzahl") 
        
        ax.set_xlim(0, max_val_per_gender * 1.15)

    plt.suptitle(f"Verteilung {column} nach Geschlecht", fontsize=16)
    plt.tight_layout()
    plt.savefig(f"../../results/charts/bar_charts/Verteilung_{column}_nach_Geschlecht", dpi=300)
    plt.close()

## Verteilung biografisches Mittel

In [12]:
# Zahlen konvertieren
df_persons['Geburtsjahr'] = pd.to_numeric(df_persons['Geburtsjahr'], errors='coerce')
df_persons['Todesjahr'] = pd.to_numeric(df_persons['Todesjahr'], errors='coerce')

# Fehlende Todesjahre schätzen
df_persons['Todesjahr'] = df_persons['Todesjahr'].fillna(df_persons['Geburtsjahr'] + 80)

# Lebensdauer / Wirkungsjahre
df_persons['lebensdauer'] = df_persons['Todesjahr'] - df_persons['Geburtsjahr']
df_persons['wirkungsjahre'] = df_persons['lebensdauer'] - 20
df_persons.loc[df_persons['wirkungsjahre'] < 0, 'wirkungsjahre'] = 0  # keine negativen Werte

# Biografisches Mittel (abgerundet)
df_persons['biografisches_mittel'] = df_persons['wirkungsjahre'].div(2).apply(lambda x: math.floor(x) if pd.notna(x) else pd.NA)

# Wirkungsjahr = Todesjahr minus biografisches Mittel
df_persons['wirkungsjahr'] = df_persons['Todesjahr'] - df_persons['biografisches_mittel']

# Jahrhundert berechnen (als ganze Zahl)
def century(jahr):
    if pd.isna(jahr):
        return pd.NA
    return int(math.ceil(jahr / 100.0))

df_persons['wirkungsjahrhundert'] = df_persons['wirkungsjahr'].apply(century)

# Nach Geschlecht trennen
men = df_persons[df_persons['Geschlecht'] == 'männlich']
women = df_persons[df_persons['Geschlecht'] == 'weiblich']

# Counts (Index als int)
century_counts_men = men['wirkungsjahrhundert'].dropna().astype(int).value_counts().sort_index()
century_counts_women = women['wirkungsjahrhundert'].dropna().astype(int).value_counts().sort_index()

# alle Jahrhunderte (Vereinigung) als sortierte int-Liste
all_centuries = sorted(set(century_counts_men.index).union(century_counts_women.index))

# Werte in korrekter Reihenfolge extrahieren
men_values = [int(century_counts_men.get(j, 0)) for j in all_centuries]
women_values = [int(century_counts_women.get(j, 0)) for j in all_centuries]

# Plot
plt.figure(figsize=(12, 6))
breite = 0.35
x = list(range(len(all_centuries)))

plt.bar([i - breite/2 for i in x], men_values, width=breite, label='Männer', color="blue")
plt.bar([i + breite/2 for i in x], women_values, width=breite, label='Frauen', color="red")

# Beschriftung auf die Balken
for i, (m_val, w_val) in enumerate(zip(men_values, women_values)):
    if m_val:
        plt.text(i - breite/2, m_val + 1, str(m_val), ha='center', va='bottom', fontsize=12)
    if w_val:
        plt.text(i + breite/2, w_val + 1, str(w_val), ha='center', va='bottom', fontsize=12)

plt.xticks(x, [str(jh) for jh in all_centuries], fontsize=12)
plt.xlabel('Jahrhundert (Wirkungsjahr nach biografischem Mittel)')
plt.ylabel('Anzahl')
plt.title('Anzahl Personen nach biografisch geschätztem Wirkungsjahrhundert und Geschlecht')
plt.legend()
plt.tight_layout()
plt.savefig("../../results/charts/bar_charts/Säulendiagramm_biografisches_Mittel.png", dpi=300)
plt.close()

### Biografisches Mittel nach Tätigkeitsfeld

In [3]:
# Zahlen konvertieren
df_persons['Geburtsjahr'] = pd.to_numeric(df_persons['Geburtsjahr'], errors='coerce')
df_persons['Todesjahr'] = pd.to_numeric(df_persons['Todesjahr'], errors='coerce')

# Fehlende Todesjahre schätzen
df_persons['Todesjahr'] = df_persons['Todesjahr'].fillna(df_persons['Geburtsjahr'] + 80)

# Lebensdauer / Wirkungsjahre
df_persons['lebensdauer'] = df_persons['Todesjahr'] - df_persons['Geburtsjahr']
df_persons['wirkungsjahre'] = df_persons['lebensdauer'] - 20
df_persons.loc[df_persons['wirkungsjahre'] < 0, 'wirkungsjahre'] = 0

# Biografisches Mittel (abgerundet)
df_persons['biografisches_mittel'] = df_persons['wirkungsjahre'] // 2

# Wirkungsjahr = Todesjahr minus biografisches Mittel
df_persons['wirkungsjahr'] = df_persons['Todesjahr'] - df_persons['biografisches_mittel']

# Jahrhundert berechnen (als ganze Zahl)
def century(jahr):
    if pd.isna(jahr):
        return pd.NA
    return int(math.ceil(jahr / 100.0))

df_persons['wirkungsjahrhundert'] = df_persons['wirkungsjahr'].apply(century)

# STEUERUNG TÄTIGKEITSFELD-FILTER
# Setze auf None, um den Filter zu deaktivieren.
# Setze auf einen String (z.B. "Maler"), um nach diesem Feld zu filtern.
filter_taetigkeitsfeld_I = "ökonomisch" 
filter_taetigkeitsfeld_II = None

# Titel und Dateinamen vorbereiten
title_parts = ["Anzahl Personen nach biografisch geschätztem Wirkungsjahrhundert und Geschlecht"]
file_suffix_parts = ["biografisches_Mittel"]

# Starte mit dem vollen DataFrame
df_plot = df_persons 

# Filter I anwenden
if filter_taetigkeitsfeld_I:
    # .str.contains() fängt den Begriff (z.B. "Maler") auch in Mehrfacheinträgen 
    # (z.B. "Maler, Bildhauer"). na=False behandelt leere Felder als "kein Treffer".
    df_plot = df_plot[df_plot['Tätigkeitsfeld'].str.contains(filter_taetigkeitsfeld_I, na=False)]
    
    # Titel und Dateiname anpassen
    title_parts.append(f"Tätigkeitsfeld: {filter_taetigkeitsfeld_I}")
    file_suffix_parts.append(filter_taetigkeitsfeld_I.replace(" ", "_")) # Leerzeichen für Dateinamen ersetzen

# Filter II anwenden
if filter_taetigkeitsfeld_II:
    df_plot = df_plot[df_plot['Tätigkeitsfeld II'].str.contains(filter_taetigkeitsfeld_II, na=False)]
    
    # Titel und Dateiname anpassen
    title_parts.append(f"T-Feld II: {filter_taetigkeitsfeld_II}")
    file_suffix_parts.append(filter_taetigkeitsfeld_II.replace(" ", "_"))

# Kombiniere Titel und Dateinamen
plot_title = " | ".join(title_parts)
file_name = f"../../results/charts/bar_charts/Säulendiagramm_{'_'.join(file_suffix_parts)}.png"


# Nach Geschlecht trennen (vom gefilterten df_plot)
men = df_plot[df_plot['Geschlecht'] == 'männlich']
women = df_plot[df_plot['Geschlecht'] == 'weiblich']

# Counts (Index als int)
century_counts_men = men['wirkungsjahrhundert'].dropna().astype(int).value_counts().sort_index()
century_counts_women = women['wirkungsjahrhundert'].dropna().astype(int).value_counts().sort_index()

# alle Jahrhunderte (Vereinigung) als sortierte int-Liste
all_centuries = sorted(set(century_counts_men.index).union(century_counts_women.index))

# Werte in korrekter Reihenfolge extrahieren
men_values = [century_counts_men.get(j, 0) for j in all_centuries]
women_values = [century_counts_women.get(j, 0) for j in all_centuries]

# Plot
plt.figure(figsize=(12, 6))
breite = 0.35
x = np.arange(len(all_centuries))

bar1 = plt.bar(x - breite/2, men_values, width=breite, label='Männer', color="blue")
bar2 = plt.bar(x + breite/2, women_values, width=breite, label='Frauen', color="red")

plt.bar_label(bar1, padding=3, fontsize=12)
plt.bar_label(bar2, padding=3, fontsize=12)

plt.xticks(x, [f"{jh}." for jh in all_centuries], fontsize=12)
plt.xlabel('Jahrhundert (Wirkungsjahr nach biografischem Mittel)')
plt.ylabel('Anzahl')

plt.title(plot_title, wrap=True)

plt.legend()
plt.tight_layout()
plt.savefig(file_name, dpi=300)
plt.close()